In [114]:
# !pip install pandas
# !pip install numpy
# !pip install seaborn
# !pip install scikit-learn
# !pip install pathlib
# !pip install googletrans
!pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/8c/3a/c9c5d4d5c49b132ef15ac7b5ccf56ef1c82efe36cd19414771762e97c00e/xgboost-1.7.6-py3-none-manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 2.3 MB/s eta 0:00:0000:0100:01


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import string
import os
import requests

from requests import get
from bs4 import BeautifulSoup
from warnings import warn
from time import sleep
from random import randint
from xgboost import XGBClassifier

from pathlib import Path
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV
from nltk.stem.porter import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from tensorflow.keras import layers, Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

2023-08-09 09:18:23.346591: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-09 09:18:23.476035: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-09 09:18:23.477453: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-09 09:18:24.953489: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## LOAD DATA

In [22]:
folder_path = '/home/jessica/code/chooj202/movie_genre_prediction/raw_data/csv/'
dataframes = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        dataframes.append(df)
merged_df = pd.concat(dataframes, ignore_index=True)

In [24]:
merged_df = merged_df.drop(columns = ['Unnamed: 0', 'movie', 'image_url'])

In [25]:
merged_df = merged_df.astype(str).drop_duplicates()

In [27]:
merged_df["genre"] = (
    merged_df["genre"]
    .apply(eval)
    .apply(lambda x: [genre.strip() for genre in x])
    )

In [28]:
merged_df

,imdb_id,genre,plot
0,tt0259534,"[Animation, Action, Adventure]","The fantastic story of Rama, a young prince wh..."
1,tt0167260,"[Action, Adventure, Drama]",The final confrontation between the forces of ...
2,tt9362722,"[Animation, Action, Adventure]",Miles Morales returns for the next chapter of ...
3,tt0120737,"[Action, Adventure, Drama]",An ancient Ring thought lost for centuries has...
4,tt0167261,"[Action, Adventure, Drama]",The continuing quest of Frodo and the Fellowsh...
...,...,...,...
5007,tt0109579,"[Drama, Mystery, Thriller]",Paulina Escobar is a political activist whose ...
5008,tt10341034,"[Drama, Mystery, Thriller]",Matthieu is a young and talented black box ana...
5010,tt1073241,"[Crime, Drama, Mystery]",Thinking Pulitzer Prize and hoping to bring do...
5012,tt0478134,"[Crime, Drama, Mystery]",A retired military investigator works with a p...


## DATA PREPROCESSING

In [29]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(merged_df['genre'])

#transform target variable
y = multilabel_binarizer.transform(merged_df['genre'])
genre_names = multilabel_binarizer.classes_

# Adding 
for i in range(len(genre_names)):
    merged_df[f"{genre_names[i]}"] = y[:,i]

merged_df.head(4)

,imdb_id,genre,plot,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,tt0259534,"[Animation, Action, Adventure]","The fantastic story of Rama, a young prince wh...",1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tt0167260,"[Action, Adventure, Drama]",The final confrontation between the forces of ...,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,tt9362722,"[Animation, Action, Adventure]",Miles Morales returns for the next chapter of ...,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0120737,"[Action, Adventure, Drama]",An ancient Ring thought lost for centuries has...,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [15]:
def cleaning(sentence):
    
    # Basic cleaning
    sentence = sentence.strip() ## remove whitespaces
    sentence = sentence.lower() ## lowercase
    sentence = ''.join(char for char in sentence if not char.isdigit()) ## remove numbers
    
    # Advanced cleaning
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, '') ## remove punctuation
    
    tokenized_sentence = word_tokenize(sentence) ## tokenize
    stop_words = set(stopwords.words('english')) ## define stopwords
    
    tokenized_sentence_cleaned = [ ## remove stopwords
    w for w in tokenized_sentence if not w in stop_words]
    
    lemmatized = [
    WordNetLemmatizer().lemmatize(word, pos = "v")
        for word in tokenized_sentence_cleaned
    ]
    
    cleaned_sentence = ' '.join(word for word in lemmatized)
    
    return cleaned_sentence

In [30]:
merged_df['plot'] = merged_df['plot'].apply(cleaning)

In [31]:
X = merged_df['plot']
y = merged_df.drop(columns = ['imdb_id', 'plot', 'genre'])

## LOGISTIC REGRESSION 

In [10]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [11]:
# Instantiating the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Training it on the train and validation set
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

In [12]:
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)

In [13]:
# Fit model on train data
clf.fit(X_train_tfidf, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [14]:
# Make predictions for validation set
y_pred = clf.predict(X_val_tfidf)

In [15]:
multilabel_binarizer.inverse_transform(y_pred)[0:10]

[('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Adventure', 'Animation', 'Comedy'),
 ('Crime', 'Drama'),
 ('Drama',),
 ('Adventure', 'Family', 'Fantasy')]

In [16]:
# Evaluate performance
accuracy_score(y_val, y_pred)

0.08742434431741762

In [17]:
# predict probabilities
y_pred_prob = clf.predict_proba(X_val_tfidf)

t = 0.3 # threshold value
y_pred_new = (y_pred_prob >= t).astype(int)

# evaluate performance
accuracy_score(y_val, y_pred_new)

0.2293207800941493

## MULTINOMIAL BAYES

In [32]:
tfidf_vectorizer = TfidfVectorizer()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 0)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

mnb = MultinomialNB()
mnb_model = OneVsRestClassifier(mnb)

mnb_model.fit(X_train_tfidf, y_train)

OneVsRestClassifier(estimator=MultinomialNB())

In [33]:
y_pred = mnb_model.predict(X_val_tfidf)

In [34]:
multilabel_binarizer.inverse_transform(y_pred)[0:10]

[('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',)]

In [35]:
accuracy_score(y_val, y_pred)

0.007722007722007722

## RANDOM FOREST CLASSIFIER

In [36]:
tfidf_vectorizer = TfidfVectorizer()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 0)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

rfc = RandomForestClassifier(random_state = 42)
rfc_model = OneVsRestClassifier(rfc)

rfc_model.fit(X_train_tfidf,y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(random_state=42))

In [37]:
y_pred = rfc_model.predict(X_val_tfidf)

In [38]:
multilabel_binarizer.inverse_transform(y_pred)[0:10]

[('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',),
 ('Drama',)]

In [39]:
accuracy_score(y_val, y_pred)

0.007722007722007722

## XGBOOST

In [121]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 0)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

xgb = XGBClassifier(objective = 'multi:softprob', random_state = 42, num_class = 21)
xgb_model = OneVsRestClassifier(xgb)

xgb_model.fit(X_train_tfidf,y_train)

OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=None,
                                            num_class=21,
                                            num_parallel_tree=None,
                                            objective='multi:softprob', ...))

In [122]:
y_pred = xgb_model.predict(X_val_tfidf)
accuracy_score(y_val, y_pred)

0.6375252185608608

## SGD

In [93]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 0)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

sgd = SGDClassifier(random_state = 42)
sgd_model = OneVsRestClassifier(sgd)
sgd_model.fit(X_train_tfidf,y_train)

OneVsRestClassifier(estimator=SGDClassifier(random_state=42))

In [94]:
y_pred = sgd_model.predict(X_val_tfidf)

In [95]:
multilabel_binarizer.inverse_transform(y_pred)[0:10]

[('Action', 'Adventure', 'Drama'),
 ('Drama',),
 ('Drama', 'Fantasy', 'Mystery'),
 ('Drama',),
 (),
 ('Adventure', 'Comedy', 'Drama'),
 ('Adventure', 'Animation', 'Comedy'),
 ('Action', 'Biography', 'Crime'),
 ('Biography', 'Drama'),
 ('Adventure', 'Family', 'Fantasy')]

In [96]:
accuracy_score(y_val, y_pred)

0.6543375924680565

### SGD - Grid Search

In [110]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 0)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

param_grid = {
    'estimator__loss' : ['hinge', 'log_loss', 'squared_hinge', 'perceptron'],
    'estimator__penalty' : ['l2', 'l1', 'elasticnet'],
    'estimator__alpha' : [10**-5, 10**-3, 10**-1, 10**1, 10**2],
    'estimator__class_weight' : ['balanced', None],
    'estimator__learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive']
}

clf = OneVsRestClassifier(SGDClassifier(max_iter = 10000))
model_tuning = GridSearchCV(clf, param_grid, cv = 2)
model_tuning.fit(X_train_tfidf,y_train)

/home/jessica/.pyenv/versions/3.10.6/envs/movie_genre_prediction/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
720 fits failed out of a total of 960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
720 fits failed with the following error:
Traceback (most recent call last):
  File "/home/jessica/.pyenv/versions/3.10.6/envs/movie_genre_prediction/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/jessica/.pyenv/versions/3.10.6/envs/movie_genre_prediction/lib/python3.10/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home

GridSearchCV(cv=2,
             estimator=OneVsRestClassifier(estimator=SGDClassifier(class_weight='balanced',
                                                                   max_iter=10000)),
             param_grid={'estimator__alpha': [1e-05, 0.001, 0.1, 10, 100],
                         'estimator__class_weight': ['balanced', None],
                         'estimator__learning_rate': ['constant', 'optimal',
                                                      'invscaling',
                                                      'adaptive'],
                         'estimator__loss': ['hinge', 'log_loss',
                                             'squared_hinge', 'perceptron'],
                         'estimator__penalty': ['l2', 'l1', 'elasticnet']})

In [111]:
print(model_tuning.best_score_)
print(model_tuning.best_params_)

0.46539792387543255
{'estimator__alpha': 1e-05, 'estimator__class_weight': 'balanced', 'estimator__learning_rate': 'optimal', 'estimator__loss': 'log_loss', 'estimator__penalty': 'l2'}


## SVC

In [99]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 0)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

In [101]:
sgd = SGDClassifier()
lr = LogisticRegression(C=1.0)
svc = SVC(kernel='linear')
nb = MultinomialNB()

# One vs Restclassifier

model = OneVsRestClassifier(svc)
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_val_tfidf)
print(f'Accuracy score for SVC linear model is {accuracy_score(y_val, y_pred)}')

Accuracy score for SVC linear model is 0.5406859448554135


In [98]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 0)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

sgd = SGDClassifier()
lr = LogisticRegression(C=1.0)
svc = SVC(kernel='sigmoid')
nb = MultinomialNB()

# One vs Restclassifier

model = OneVsRestClassifier(svc)
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_val_tfidf)
print(f'Accuracy score for SVC sigmoid model is {accuracy_score(y_val, y_pred)}')

Accuracy score for SVC model sigmoid is 0.429724277067922


In [103]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 0)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

sgd = SGDClassifier()
lr = LogisticRegression(C=1.0)
svc = SVC(kernel='poly')
nb = MultinomialNB()

# One vs Restclassifier

model = OneVsRestClassifier(svc)
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_val_tfidf)
print(f'Accuracy score for SVC poly model is {accuracy_score(y_val, y_pred)}')

Accuracy score for SVC poly model is 0.6355077336919973


## NAIVE BAYES

In [84]:
# One vs Restclassifier
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)

model = OneVsRestClassifier(nb)
model.fit(X_train_tfidf, y_train)
y_pred = model.predict(X_val_tfidf)
print(f'Accuracy score for Naive Bayes model is {accuracy_score(y_val, y_pred)}')

Accuracy score for Naive Bayes model is 0.04169468728984533


## RNN MODEL

In [18]:
X2 = merged_df['plot']
y2 = merged_df.drop(columns = ['movie', 'imdb_id', 'plot', 'genre', 'Unnamed: 0', 'image_url'])

In [19]:
tk = Tokenizer()
tk.fit_on_texts(X2)
vocab_size = len(tk.word_index)
print(f'There are {vocab_size} different words in your corpus')
X2_token = tk.texts_to_sequences(X2)

X2_pad = pad_sequences(X2_token, dtype = 'float32', padding = 'post')

There are 21368 different words in your corpus


In [20]:
multilabel_binarizer.classes_.shape

(21,)

In [21]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2_pad, y2, test_size = 0.3, random_state = 0)

In [22]:
model = Sequential()
model.add(layers.Embedding(
    input_dim = vocab_size +1,
    input_length = 292,
    output_dim = 100,
    mask_zero = True))

model.add(layers.Dense(1024, activation = 'relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(1505, activation = 'relu'))
model.add(layers.LSTM(20))
model.add(layers.Dense(21, activation='softmax'))

In [23]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics = 'accuracy')

model.fit(X2_train, y2_train, epochs = 5, batch_size = 16, verbose = 1, validation_split = 0.3)

Epoch 1/5


2023-08-06 16:48:52.854133: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19136512 exceeds 10% of free system memory.
2023-08-06 16:48:53.031268: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19136512 exceeds 10% of free system memory.
2023-08-06 16:48:53.031397: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19136512 exceeds 10% of free system memory.
2023-08-06 16:48:53.112892: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28125440 exceeds 10% of free system memory.
2023-08-06 16:48:53.523424: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28125440 exceeds 10% of free system memory.


152/152 [==============================] - 110s 650ms/step - loss: 8.3164 - accuracy: 0.1743 - val_loss: 8.3502 - val_accuracy: 0.1710
Epoch 2/5
152/152 [==============================] - 91s 600ms/step - loss: 8.4657 - accuracy: 0.1759 - val_loss: 8.4994 - val_accuracy: 0.1710
Epoch 3/5
152/152 [==============================] - 91s 599ms/step - loss: 8.5843 - accuracy: 0.1759 - val_loss: 8.6044 - val_accuracy: 0.1710
Epoch 4/5
152/152 [==============================] - 93s 612ms/step - loss: 8.7090 - accuracy: 0.1759 - val_loss: 8.7789 - val_accuracy: 0.1710
Epoch 5/5
152/152 [==============================] - 91s 599ms/step - loss: 8.9402 - accuracy: 0.1759 - val_loss: 8.9395 - val_accuracy: 0.1710


In [24]:
model.evaluate(X2_test, y2_test, verbose = 0)

[9.015741348266602, 0.18964357674121857]

TTD

- try count vectoriser
- descriptive statistics
- accuracy/f1 for each individual genre

In [128]:
!git add Basic-Model.ipynb

In [129]:
!git commit -m "tried a few models"

[jess_new_branch a88738b] tried a few models
 1 file changed, 1670 insertions(+)
 create mode 100644 notebooks/Basic-Model.ipynb


In [130]:
!git push origin jess_new_branch

Enumerating objects: 6, done.
Counting objects: 100% (6/6), done.
Delta compression using up to 12 threads
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 12.69 KiB | 12.69 MiB/s, done.
Total 4 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To github.com:chooj202/movie_genre_prediction.git
   9ecf88c..a88738b  jess_new_branch -> jess_new_branch
